# Lora 实战

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "5"

## Step1 导入相关包

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("../data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [4]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

In [5]:
print(len("以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。"))

207


## Step3 数据集预处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("/data/PLM/Llama-2-7b-ms")
tokenizer # 可以看到默认的padding_side是left

LlamaTokenizerFast(name_or_path='/data/PLM/Llama-2-7b-ms', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [7]:
# 特殊token的id是这些，别被上面什么0,1,2,32000误导了！
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.unk_token_id)
print(tokenizer.pad_token_id)
print(tokenizer.cls_token_id)

1
2
0
0
None


In [8]:
# 训练阶段最好设置padding_side为right，因为有些tokenizer使用的是绝对位置编码，导致非pad token的position id不是从0开始的
# 实验过程中也发现padding_side为left可能导致loss剧烈波动的现象
tokenizer.padding_side = "right" 
# pad_token_id = eos_token_id的风险在于如果推理阶段也使用right padding，那decoder-only的模型需要从最后一个token的probability中sample下一个token，会出现问题
# 所以推理阶段要么不用padding，要么用left padding
tokenizer.pad_token_id = tokenizer.eos_token_id # pad_token_id最好不要设置成0，不然训练过程中会出现大量nan的溢出，导致最终的training loss长期为0也无法收敛！！

In [9]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    # [CLS]在LLaMA分词器中会自带，表示为<s>，可以通过add_special_tokens=False去掉，因为decoder-only的模型并不需要[CLS]等开头的token来做额外的处理；这里也还没有padding
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ", add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    # [EOS]在LLaMA分词器中不会自带，即使add_special_tokens=True，必须手动添加，表示为</s>
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    # 果然！labels中input的部分也需要改成-100！
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    # 这种截断方式会舍弃最后的[EOS]，可以修改一下
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [10]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Map: 100%|██████████| 26858/26858 [00:14<00:00, 1800.13 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [11]:
print(tokenized_ds[0]["input_ids"])

[12968, 29901, 29871, 30982, 31695, 31863, 31577, 30210, 30457, 30502, 31302, 30858, 30267, 13, 13, 7900, 22137, 29901, 29871, 29871, 30651, 30557, 30392, 30982, 31695, 31863, 31577, 30210, 30457, 30502, 31302, 30858, 30383, 13, 13, 29896, 29889, 29871, 30982, 31695, 31687, 30988, 31704, 30846, 30267, 31951, 30408, 232, 132, 157, 236, 131, 133, 30948, 30210, 31687, 30988, 31894, 30846, 30214, 30847, 233, 152, 166, 233, 176, 168, 30330, 235, 186, 148, 233, 176, 168, 31391, 233, 187, 187, 233, 182, 182, 30214, 30815, 231, 194, 134, 31174, 30869, 235, 164, 131, 31624, 31863, 31577, 30214, 232, 165, 161, 232, 191, 189, 235, 133, 143, 235, 133, 140, 31074, 31180, 30214, 31666, 30417, 31931, 30909, 232, 138, 146, 31022, 30988, 30908, 30267, 13, 13, 29906, 29889, 29871, 232, 160, 138, 235, 164, 164, 236, 168, 177, 31855, 30267, 31951, 30408, 31855, 30406, 30374, 236, 181, 159, 30210, 235, 151, 175, 31854, 30330, 30716, 30801, 30330, 30753, 31112, 30834, 30503, 235, 135, 133, 235, 133, 173, 23

In [12]:
# 注意这两者的区别！LLaMA的tokenizer会把直接加上的eos_token再次分词（如<\s> = <\ + s + >）。解决办法是要么像这样先加个空格，要么encode后再加eos_token_id！
print(tokenizer("abc" + tokenizer.eos_token))
print(tokenizer("abc " + tokenizer.eos_token))

{'input_ids': [1, 25638, 829, 29879, 29958], 'attention_mask': [1, 1, 1, 1, 1]}
{'input_ids': [1, 25638, 2], 'attention_mask': [1, 1, 1]}


In [13]:
tokenizer.decode(tokenized_ds[0]["input_ids"])

'Human: 保持健康的三个提示。\n\nAssistant:  以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

In [14]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## Step4 创建模型

In [15]:
import torch
# 仅使用torch_dtype=torch.half并不经过量化和反量化过程！只是改变了权重的范围！
model = AutoModelForCausalLM.from_pretrained("/data/PLM/Llama-2-7b-ms", low_cpu_mem_usage=True, torch_dtype=torch.half, device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.53s/it]
/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization

In [16]:
model.dtype

torch.float16

## Lora

### PEFT Step1 配置文件

In [17]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM,)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

### PEFT Step2 创建模型

In [18]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/data/PLM/Llama-2-7b-ms', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'v_proj'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})

In [19]:
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [20]:
for name, parameter in model.named_parameters(): # 只有qv需要Lora，k是不需要的
    print(name, parameter.dtype) # 可以看到被冻结的部分权重都是fp16不变，但Lora的部分权重仍是fp32

base_model.model.model.embed_tokens.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.weight torch.float16
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.float32
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.float32
base_model.model.model.layers.0.self_attn.k_proj.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.weight torch.float16
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.float32
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.float32
base_model.model.model.layers.0.self_attn.o_proj.weight torch.float16
base_model.model.model.layers.0.mlp.gate_proj.weight torch.float16
base_model.model.model.layers.0.mlp.up_proj.weight torch.float16
base_model.model.model.layers.0.mlp.down_proj.weight torch.float16
base_model.model.model.layers.0.input_layernorm.weight torch.float16
base_model.model.model.layers.0.post_attention_layernor

In [21]:
# model = model.half() # 通过这种方式将Lora也改成fp16
torch.tensor(1e-8).half() # 当整个模型都是半精度时，需要将adam_epsilon调大，因为fp16的最小值只有5.96×1e-8，会导致小于这个值的数都变成0，从而导致除0错误！

tensor(0., dtype=torch.float16)

In [22]:
model.enable_input_require_grads() # Lora情况下只有开启了这个才能实现梯度检查点

In [23]:
from torch.utils.data import DataLoader
exampled = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)) # 每个批次都根据其内部最长的序列来决定填充长度
exampled_ipt = next(enumerate(exampled))[1] # 这是在劝我们编码的时候就把padding弄好呢，不过不用管它
exampled_ipt

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[12968, 29901, 29871, 30982, 31695, 31863, 31577, 30210, 30457, 30502,
         31302, 30858, 30267,    13,    13,  7900, 22137, 29901, 29871, 29871,
         30651, 30557, 30392, 30982, 31695, 31863, 31577, 30210, 30457, 30502,
         31302, 30858, 30383,    13,    13, 29896, 29889, 29871, 30982, 31695,
         31687, 30988, 31704, 30846, 30267, 31951, 30408,   232,   132,   157,
           236,   131,   133, 30948, 30210, 31687, 30988, 31894, 30846, 30214,
         30847,   233,   152,   166,   233,   176,   168, 30330,   235,   186,
           148,   233,   176,   168, 31391,   233,   187,   187,   233,   182,
           182, 30214, 30815,   231,   194,   134, 31174, 30869,   235,   164,
           131, 31624, 31863, 31577, 30214,   232,   165,   161,   232,   191,
           189,   235,   133,   143,   235,   133,   140, 31074, 31180, 30214,
         31666, 30417, 31931, 30909,   232,   138,   146, 31022, 30988, 30908,
         30267,    13,    13, 29906, 2

In [24]:
model(**exampled_ipt.to("cuda"))

CausalLMOutputWithPast(loss=tensor(1.2301, device='cuda:0', grad_fn=<ToCopyBackward0>), logits=tensor([[[-10.5938,  -6.6289,  -0.1225,  ...,  -7.9180,  -8.1406,  -9.6953],
         [-13.7031, -12.3438,   0.6689,  ...,  -8.9766,  -9.5547, -10.0703],
         [ -7.3711,  -5.4492,   2.3320,  ...,   3.4902,  -0.0424,   0.4314],
         ...,
         [ -4.5430,  -1.4082,  11.8359,  ...,   8.3203,   2.3711,   7.2500],
         [  0.3655,   3.7539,  18.0625,  ...,   8.9141,   6.4648,  11.3828],
         [  2.5195,  31.1406,   2.1855,  ...,   3.7188,   1.9346,   2.2188]],

        [[-10.5938,  -6.6289,  -0.1225,  ...,  -7.9180,  -8.1406,  -9.6953],
         [-13.7031, -12.3438,   0.6689,  ...,  -8.9766,  -9.5547, -10.0703],
         [ -7.3711,  -5.4492,   2.3320,  ...,   3.4902,  -0.0424,   0.4314],
         ...,
         [  1.6494,  32.5938,   2.5664,  ...,   3.2871,   1.6553,   2.6484],
         [  1.6045,  32.7188,   2.5996,  ...,   3.2812,   1.6602,   2.6387],
         [  1.5889,  32.7812

## Step5 配置训练参数

In [25]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    gradient_checkpointing=True, # 设置梯度检查点，在神经网络训练过程中动态计算和存储最小层数以节约显存。代价是需要额外的计算时间(约20%)
    # adam_epsilon=6e-8, # 避免半精度下的除0问题，起因是fp16下非零最小值只到5.96×1e-8，所以默认的1e-8也会变成0！
)

## Step6 创建训练器

In [26]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True), # label_pad_token_id参数控制用于标签填充的token id，默认值是-100（太好了）
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step7 模型训练

In [27]:
trainer.train() #  6000/375=16=8*2，也就是batch样本数*梯度累计

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


TrainOutput(global_step=375, training_loss=1.0350398089090984, metrics={'train_runtime': 1190.4383, 'train_samples_per_second': 5.04, 'train_steps_per_second': 0.315, 'total_flos': 5.085975792648192e+16, 'train_loss': 1.0350398089090984, 'epoch': 1.0})

## Step8 模型推理

In [28]:
model.eval()
# model.generate的时候最好不要padding，就算为了加速推理也只能用left padding！
ipt = tokenizer("Human: {}\n{}".format("你好", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device) # 用model.device可以避免繁琐的重新设置，直接转移到相应的设备上
tokenizer.decode(model.generate(**ipt, max_length=512, do_sample=True, eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'Human: 你好\n\nAssistant:  您好。我是一个受到人类熟视和使用的电脑助手。我的人工智能可以帮助您解决问题和提供相关服务。您可以与我进行对话，提供一些信息，就像是交谈对话那样。您可以说多大语言？我能说华语和英语。而且，我现在正在学中文。因此，我会不断提升自己的语言能力以便帮助您。'